In [2]:
#load package
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler, SMOTE
from sklearn.datasets import make_classification

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV

from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

import warnings
# Ignore all warnings
warnings.filterwarnings("ignore")

In [5]:
# path of the file
file_name = 'D:/Liuuu/Graduate@UCSD/Intern/DS/CapitalOne/transactions/processed_transactions.csv'
transactions=pd.read_csv(file_path)
transactions.head(10)

,accountNumber,customerId,creditLimit,availableMoney,transactionDateTime,transactionAmount,merchantName,acqCountry,merchantCountryCode,posEntryMode,posConditionCode,merchantCategoryCode,currentExpDate,accountOpenDate,dateOfLastAddressChange,cardCVV,enteredCVV,cardLast4Digits,transactionType,currentBalance,cardPresent,expirationDateKeyInMatch,isFraud,Date
0,737265056,737265056,5000,5000.00,2016-08-13 14:27:32,98.55,Uber,US,US,02,01,rideshare,06/2023,2015-03-14,2015-03-14,414,414,1803,PURCHASE,0.00,False,False,False,2016-08-13
1,737265056,737265056,5000,5000.00,2016-10-11 05:05:54,74.51,AMC #191138,US,US,09,01,entertainment,02/2024,2015-03-14,2015-03-14,486,486,767,PURCHASE,0.00,True,False,False,2016-10-11
2,737265056,737265056,5000,5000.00,2016-11-08 09:18:39,7.47,Play Store,US,US,09,01,mobileapps,08/2025,2015-03-14,2015-03-14,486,486,767,PURCHASE,0.00,False,False,False,2016-11-08
3,737265056,737265056,5000,5000.00,2016-12-10 02:14:50,7.47,Play Store,US,US,09,01,mobileapps,08/2025,2015-03-14,2015-03-14,486,486,767,PURCHASE,0.00,False,False,False,2016-12-10
4,830329091,830329091,5000,5000.00,2016-03-24 21:04:46,71.18,Tim Hortons #947751,US,US,02,01,fastfood,10/2029,2015-08-06,2015-08-06,885,885,3143,PURCHASE,0.00,True,False,False,2016-03-24
5,830329091,830329091,5000,5000.00,2016-04-19 16:24:27,30.76,In-N-Out #422833,US,US,02,01,fastfood,01/2020,2015-08-06,2015-08-06,885,885,3143,PURCHASE,0.00,True,False,False,2016-04-19
6,830329091,830329091,5000,5000.00,2016-05-21 14:50:35,57.28,Krispy Kreme #685312,US,US,02,01,fastfood,05/2020,2015-08-06,2015-08-06,885,885,3143,PURCHASE,0.00,True,False,False,2016-05-21
7,830329091,830329091,5000,5000.00,2016-06-03 00:31:21,9.37,Shake Shack #968081,US,US,05,01,fastfood,01/2021,2015-08-06,2015-08-06,885,885,3143,PURCHASE,0.00,True,False,False,2016-06-03
8,830329091,830329091,5000,4990.63,2016-06-10 01:21:46,523.67,Burger King #486122,,US,02,01,fastfood,08/2032,2015-08-06,2015-08-06,885,885,3143,PURCHASE,9.37,True,False,False,2016-06-10
9,830329091,830329091,5000,5000.00,2016-07-11 10:47:16,164.37,Five Guys #510989,US,US,05,08,fastfood,04/2020,2015-08-06,2015-08-06,885,885,3143,PURCHASE,0.00,True,False,False,2016-07-11


In [6]:
transactions["isFraud"].isna().sum()

0

In [7]:
fraudulent_records = transactions[transactions['isFraud'] == True]
fraudulent_records.head()

,accountNumber,customerId,creditLimit,availableMoney,transactionDateTime,transactionAmount,merchantName,acqCountry,merchantCountryCode,posEntryMode,posConditionCode,merchantCategoryCode,currentExpDate,accountOpenDate,dateOfLastAddressChange,cardCVV,enteredCVV,cardLast4Digits,transactionType,currentBalance,cardPresent,expirationDateKeyInMatch,isFraud,Date
694,984504651,984504651,50000,8613.80,2016-09-08 16:14:40,165.26,Walgreens #860120,US,US,09,08,health,10/2021,2015-07-27,2016-06-25,640,640,8332,PURCHASE,41386.20,False,False,True,2016-09-08
971,114896048,114896048,15000,14113.96,2016-01-06 08:05:06,298.85,sears.com,US,US,09,01,online_retail,09/2028,2015-12-19,2015-12-19,773,773,8915,PURCHASE,886.04,False,False,True,2016-01-06
1072,114896048,114896048,15000,13702.47,2016-03-07 02:40:54,406.89,Planet Fitness #136615,US,US,09,01,health,05/2028,2015-12-19,2015-12-19,773,773,8915,PURCHASE,1297.53,False,False,True,2016-03-07
1113,114896048,114896048,15000,8943.58,2016-04-09 15:14:04,439.31,oldnavy.com,US,US,09,01,online_retail,05/2032,2015-12-19,2015-12-19,773,773,8915,PURCHASE,6056.42,False,False,True,2016-04-09
1321,114896048,114896048,15000,5223.03,2016-08-24 01:24:35,266.09,sears.com,US,US,09,01,online_retail,05/2028,2015-12-19,2016-06-10,773,773,8915,PURCHASE,9776.97,False,False,True,2016-08-24


In [8]:
fraudulent_records.shape

(12417, 24)

## Data Preprocessing for Machine Learning

In [246]:
## Create X and Y
## Add Feature 'total_seconds'
transactions['time_diff'] = transactions.groupby('accountNumber')['transactionDateTime'].diff().fillna(pd.Timedelta(0))
transactions_sorted = transactions.sort_values(by='time_diff', ascending=False)
transactions_sorted['total_seconds'] = transactions_sorted['time_diff'].dt.total_seconds()

## process X
accountNumber = X['accountNumber']
X = transactions_sorted.drop(labels="isFraud", axis=1)
X["cardPresent"] = X["cardPresent"].astype(int)
X.drop('accountNumber', axis = 1, inplace = True)

## process Y
Y = transactions_sorted["isFraud"].astype(int)

In [247]:
transactions_sorted.head()

,accountNumber,customerId,creditLimit,availableMoney,transactionDateTime,transactionAmount,merchantName,acqCountry,merchantCountryCode,posEntryMode,posConditionCode,merchantCategoryCode,currentExpDate,accountOpenDate,dateOfLastAddressChange,cardCVV,enteredCVV,cardLast4Digits,transactionType,currentBalance,cardPresent,expirationDateKeyInMatch,isFraud,Date,time_diff,total_seconds
401365,454501168,454501168,7500,7500.0,2016-12-11 05:41:24,8.75,Play Store,US,US,02,01,mobileapps,01/2021,2015-10-07,2015-10-07,971,971,4824,PURCHASE,0.0,False,False,False,2016-12-11,328 days 03:50:56,28353056.0
405838,740708179,740708179,15000,15000.0,2016-11-24 05:30:31,143.99,In-N-Out #27394,US,US,05,01,fastfood,05/2028,2015-03-06,2015-03-06,630,630,9830,PURCHASE,0.0,True,False,False,2016-11-24,318 days 16:07:59,27533279.0
19121,325487838,325487838,1000,1000.0,2016-12-26 18:16:05,14.27,Spotify,US,US,09,01,online_subscriptions,07/2033,2015-05-30,2015-05-30,237,234,2739,PURCHASE,0.0,False,False,False,2016-12-26,314 days 12:58:23,27176303.0
132160,666461201,666461201,5000,5000.0,2016-11-13 08:03:00,395.31,oldnavy.com,US,US,09,01,online_retail,03/2028,2008-06-19,2008-06-19,204,204,2797,PURCHASE,0.0,False,False,True,2016-11-13,300 days 02:21:37,25928497.0
573905,951879101,951879101,10000,9827.3,2016-11-11 13:44:19,69.94,ebay.com,US,US,02,08,online_retail,12/2021,2014-08-23,2014-08-23,789,789,3796,PURCHASE,172.7,False,False,False,2016-11-11,290 days 13:48:07,25105687.0


In [248]:
X.head()

,customerId,creditLimit,availableMoney,transactionDateTime,transactionAmount,merchantName,acqCountry,merchantCountryCode,posEntryMode,posConditionCode,merchantCategoryCode,currentExpDate,accountOpenDate,dateOfLastAddressChange,cardCVV,enteredCVV,cardLast4Digits,transactionType,currentBalance,cardPresent,expirationDateKeyInMatch,Date,time_diff,total_seconds
401365,454501168,7500,7500.0,2016-12-11 05:41:24,8.75,Play Store,US,US,02,01,mobileapps,01/2021,2015-10-07,2015-10-07,971,971,4824,PURCHASE,0.0,0,False,2016-12-11,328 days 03:50:56,28353056.0
405838,740708179,15000,15000.0,2016-11-24 05:30:31,143.99,In-N-Out #27394,US,US,05,01,fastfood,05/2028,2015-03-06,2015-03-06,630,630,9830,PURCHASE,0.0,1,False,2016-11-24,318 days 16:07:59,27533279.0
19121,325487838,1000,1000.0,2016-12-26 18:16:05,14.27,Spotify,US,US,09,01,online_subscriptions,07/2033,2015-05-30,2015-05-30,237,234,2739,PURCHASE,0.0,0,False,2016-12-26,314 days 12:58:23,27176303.0
132160,666461201,5000,5000.0,2016-11-13 08:03:00,395.31,oldnavy.com,US,US,09,01,online_retail,03/2028,2008-06-19,2008-06-19,204,204,2797,PURCHASE,0.0,0,False,2016-11-13,300 days 02:21:37,25928497.0
573905,951879101,10000,9827.3,2016-11-11 13:44:19,69.94,ebay.com,US,US,02,08,online_retail,12/2021,2014-08-23,2014-08-23,789,789,3796,PURCHASE,172.7,0,False,2016-11-11,290 days 13:48:07,25105687.0


In [249]:
## Find boolean and numeric columns
# Select numeric columns
numeric_columns = X.select_dtypes(include=['number'])
# Select boolean columns
boolean_columns = X.select_dtypes(include=['bool'])

print(numeric_columns.columns)
print(boolean_columns.columns)

Index(['customerId', 'creditLimit', 'availableMoney', 'transactionAmount',
       'cardCVV', 'enteredCVV', 'cardLast4Digits', 'currentBalance',
       'cardPresent', 'time_diff', 'total_seconds'],
      dtype='object')
Index(['expirationDateKeyInMatch'], dtype='object')


## Feature Engineering

In [250]:
x_columns = ["transactionAmount", "total_seconds","posEntryMode","posConditionCode","merchantCategoryCode", "acqCountry"]
X = X[x_columns]
X_dummies = pd.get_dummies(X, columns = ["posEntryMode", "posConditionCode", "merchantCategoryCode", "acqCountry"])
X_dummies.head()
X = dummies

,transactionAmount,total_seconds,posEntryMode_,posEntryMode_02,posEntryMode_05,posEntryMode_09,posEntryMode_80,posEntryMode_90,posConditionCode_,posConditionCode_01,posConditionCode_08,posConditionCode_99,merchantCategoryCode_airline,merchantCategoryCode_auto,merchantCategoryCode_cable/phone,merchantCategoryCode_entertainment,merchantCategoryCode_fastfood,merchantCategoryCode_food,merchantCategoryCode_food_delivery,merchantCategoryCode_fuel,merchantCategoryCode_furniture,merchantCategoryCode_gym,merchantCategoryCode_health,merchantCategoryCode_hotels,merchantCategoryCode_mobileapps,merchantCategoryCode_online_gifts,merchantCategoryCode_online_retail,merchantCategoryCode_online_subscriptions,merchantCategoryCode_personal care,merchantCategoryCode_rideshare,merchantCategoryCode_subscriptions,acqCountry_,acqCountry_CAN,acqCountry_MEX,acqCountry_PR,acqCountry_US
401365,8.75,28353056.0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
405838,143.99,27533279.0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
19121,14.27,27176303.0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
132160,395.31,25928497.0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
573905,69.94,25105687.0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1


## Build Machine Learning Model

### Logistic Regression

In [255]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the machine learning pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),                            # Step 1: Standardize the features
    ('classifier', LogisticRegression())                    # Step 2: Train a Logistic Regression classifier
])

# Define the parameter grid to search
param_grid = {
    'classifier__penalty': ['l1', 'l2'],                    # Regularization penalty options
    'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100],       # Regularization parameter values
    'classifier__solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],  # Solver options
    'classifier__max_iter': [100, 500, 1000],               # Maximum number of iterations
    'classifier__random_state': [42]                       # Random state for reproducibility
}

# Create the grid search object
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy')

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Make predictions on the test set using the best model
y_pred = grid_search.predict(X_test)

# evaluation
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

conf_matrix = confusion_matrix(y_test, y_pred)
print(f"Confusion Matrix:\n{conf_matrix}")

class_report = classification_report(y_test, y_pred)
print(f"Classification Report:\n{class_report}")

Accuracy: 0.665391584457419
Confusion Matrix:
[[1595  890]
 [ 772 1710]]
Classification Report:
              precision    recall  f1-score   support

           0       0.67      0.64      0.66      2485
           1       0.66      0.69      0.67      2482

    accuracy                           0.67      4967
   macro avg       0.67      0.67      0.67      4967
weighted avg       0.67      0.67      0.67      4967



### XGBoost

In [280]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Create a data pipeline with XGBoost classifier
pipeline = Pipeline([
    ('scaler', StandardScaler()),                 # Step 1: Standardize the features
    ('classifier', XGBClassifier())               # Step 2: Train an XGBoost classifier
])

# Define the hyperparameter search space
param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5],
    'min_child_weight': [1, 2, 3],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0],
    'n_estimators': [100, 200, 300],
    'scale_pos_weight': [1, 2, 3]
}

# Use GridSearchCV for cross-validation
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Make predictions on the test set using the best model
y_pred = grid_search.predict(X_test)

# Output performance metrics for the best model
accuracy = accuracy_score(y_test, y_pred)
print(f"Best Model Accuracy: {accuracy}")

print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Best Model Accuracy: 0.6819005435876787
Classification Report:
               precision    recall  f1-score   support

           0       0.69      0.68      0.68      2507
           1       0.68      0.68      0.68      2460

    accuracy                           0.68      4967
   macro avg       0.68      0.68      0.68      4967
weighted avg       0.68      0.68      0.68      4967

Confusion Matrix:
 [[1712  795]
 [ 785 1675]]


### Random Forest

In [326]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Create a data pipeline with RandomForestClassifier
pipeline = Pipeline([
    ('scaler', StandardScaler()),                         # Step 1: Standardize the features
    ('classifier', RandomForestClassifier())               # Step 2: Train a RandomForest classifier
])

param_grid = {
    'classifier__n_estimators': [50, 100, 200, 300, 500],
    'classifier__criterion': ['gini', 'entropy'],
    'classifier__max_depth': [5, 10, 16, 20],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [1, 2, 4],
    'classifier__max_features': ['sqrt', 'log2'],
    'classifier__bootstrap': [True, False],
    'classifier__random_state': [42]
}

# Use GridSearchCV for cross-validation
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Make predictions on the test set using the best model
y_pred = grid_search.predict(X_test)

# Evaluate model performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Output other evaluation metrics
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.6853231326756594
Classification Report:
               precision    recall  f1-score   support

           0       0.68      0.70      0.69      2507
           1       0.69      0.67      0.68      2460

    accuracy                           0.69      4967
   macro avg       0.69      0.69      0.69      4967
weighted avg       0.69      0.69      0.69      4967

Confusion Matrix:
 [[1751  756]
 [ 807 1653]]


## Complete Machine Learning Pipeline (includes selecting best model from Random Forest, XGBoost, Logistic Regression)

In [24]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a data pipeline with classifiers and their parameter grids
classifiers = {
    'Random Forest': (RandomForestClassifier(), {
        'classifier__n_estimators': [50, 100, 200, 300, 500],
        'classifier__criterion': ['gini', 'entropy'],
        'classifier__max_depth': [5, 10, 16, 20],
        'classifier__min_samples_split': [2, 5, 10],
        'classifier__min_samples_leaf': [1, 2, 4],
        'classifier__max_features': ['sqrt', 'log2'],
        'classifier__bootstrap': [True, False],
        'classifier__random_state': [42]
    }),
    'XGBoost': (XGBClassifier(), {
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 4, 5],
        'min_child_weight': [1, 2, 3],
        'subsample': [0.8, 0.9, 1.0],
        'colsample_bytree': [0.8, 0.9, 1.0],
        'n_estimators': [100, 200, 300],
        'scale_pos_weight': [1, 2, 3]
    }),
    'Logistic Regression': (LogisticRegression(), {
        'classifier__penalty': ['l1', 'l2'],                    # Regularization penalty options
        'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100],       # Regularization parameter values
        'classifier__solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],  # Solver options
        'classifier__max_iter': [100, 500, 1000],               # Maximum number of iterations
        'classifier__random_state': [42]                       # Random state for reproducibility
    })
}

# Train and evaluate each classifier with grid search
best_model = None
best_accuracy = 0

for name, (clf, param_grid) in classifiers.items():
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('classifier', clf)
    ])

    # Grid search
    grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy')
    grid_search.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = grid_search.predict(X_test)

    # Evaluate the performance
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Model: {name}, Accuracy: {accuracy:.2f}")

    # Update best model if the current model has higher accuracy
    if accuracy > best_accuracy:
        Best_Model = name
        best_accuracy = accuracy
        best_model = grid_search.best_estimator_

# Display the results of the best model
print(f"\nBest Model: {Best_Model}")
print(f"Best Accuracy: {best_accuracy:.1f}")

Model: Random Forest, Accuracy: 0.69
Model: XGBoost, Accuracy: 0.68
Model: Logistic Regression, Accuracy: 0.67

Best Model: Random Forest
Best Accuracy: 0.7
